In [2]:
from google.colab import userdata
import os

# 1. Récupération des secrets
try:
    token = userdata.get('GITHUB_TOKEN')
except ImportError:
    print("Erreur: Assurez-vous d'avoir ajouté le secret GITHUB_TOKEN dans la barre latérale.")
    token = input("Entrez votre token manuellement: ")

# 2. Configuration des variables
username = "Thorbjornen"
repo_owner = "esmeml" # Le compte qui possède le repo
repo_name = "parliament_speaker_diarisation"
user_email = "niels.groeninck4@gmail.com"

# 3. Configuration de l'identité Git (nécessaire pour commit)
!git config --global user.email "{user_email}"
!git config --global user.name "{username}"

# 4. Clonage du dépôt avec authentification
# La syntaxe est : https://<token>@github.com/<owner>/<repo>.git
clone_url = f"https://{token}@github.com/{repo_owner}/{repo_name}.git"

# On clone uniquement si le dossier n'existe pas déjà
if not os.path.exists(repo_name):
    !git clone {clone_url}
    print(f"Dépôt {repo_name} cloné avec succès !")
else:
    print(f"Le dossier {repo_name} existe déjà.")

# 5. Se déplacer dans le dossier du dépôt
%cd {repo_name}

Cloning into 'parliament_speaker_diarisation'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 2), reused 6 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 1.24 MiB | 5.69 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Dépôt parliament_speaker_diarisation cloné avec succès !
/content/parliament_speaker_diarisation


In [ ]:
# Installation des librairies nécessaires pour le traitement audio et le XML
!pip install librosa soundfile lxml tensorflow

import os
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
import xml.etree.ElementTree as ET
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import drive

# 1. Connexion au Google Drive
drive.mount('/content/drive')
print("Google Drive connecté.")

In [ ]:
AUDIO_FILE_NAME = '4_decembre.wav'
XML_FILE_NAME = '4_decembre.xml'

# Chemins (Racine du Drive)
BASE_DIR = '/content/drive/MyDrive'
AUDIO_PATH = os.path.join(BASE_DIR, AUDIO_FILE_NAME)
XML_PATH = os.path.join(BASE_DIR, XML_FILE_NAME)

# Dossier où seront créés les petits segments audio
OUTPUT_DATASET_DIR = os.path.join(BASE_DIR, 'Dataset_Assemblee_CTC')
CSV_MANIFEST_PATH = os.path.join(OUTPUT_DATASET_DIR, 'metadata.csv')

# Paramètres Audio pour CTC
TARGET_SR = 16000  # Standard pour la reconnaissance vocale (16kHz)

# Création du dossier de sortie s'il n'existe pas
os.makedirs(OUTPUT_DATASET_DIR, exist_ok=True)

print(f"Dossier de travail : {OUTPUT_DATASET_DIR}")
if not os.path.exists(AUDIO_PATH):
    print(f"ERREUR : Le fichier audio '{AUDIO_FILE_NAME}' est introuvable à la racine du Drive !")
else:
    print(f"Fichier audio détecté : {AUDIO_PATH}")

In [ ]:
import xml.etree.ElementTree as ET
import json
import os
import re

# Config
XML_FILE = '/content/drive/MyDrive/4_decembre.xml'
JSON_OUTPUT = '/content/drive/MyDrive/segments_clean.json'

def xml_to_json_segments(xml_path):
    print(f"Lecture du fichier : {xml_path}")

    with open(xml_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # On retire les namespaces
    content = re.sub(r' xmlns="[^"]+"', '', content)

    root = ET.fromstring(content)

    raw_segments = []

    # On utilise .iter() pour aller chercher l'info partout
    all_paras = list(root.iter('paragraphe'))
    print(f"Nombre de paragraphes trouvés : {len(all_paras)}")

    for para in all_paras:
        texte_node = para.find('texte')

        if texte_node is not None and 'stime' in texte_node.attrib:
            try:
                start_time = float(texte_node.attrib['stime'])

                # 1. Extraction brute de tout le texte (y compris dans les balises enfants)
                raw_text = "".join(texte_node.itertext())

                # 2. Enlever les \n
                clean_text = " ".join(raw_text.split())

                # 3. Chercher le locuteur
                speaker = "Inconnu"
                orateurs = para.find('orateurs')
                if orateurs is not None:
                    orateur = orateurs.find('orateur')
                    if orateur is not None:
                        nom = orateur.find('nom')
                        if nom is not None:
                            speaker = nom.text

                if clean_text:
                    raw_segments.append({
                        'start': start_time,
                        'speaker': speaker,
                        'text': clean_text
                    })

            except ValueError:
                continue

    # Tri et calcul des durées
    raw_segments.sort(key=lambda x: x['start'])

    final_dataset = []

    for i in range(len(raw_segments) - 1):
        curr = raw_segments[i]
        nxt = raw_segments[i+1]

        duration = nxt['start'] - curr['start']

        # Filtres de qualité
        # On garde si c'est raisonnable (entre 0.5s et 60s)
        # On augmente un peu la tolérance car les applaudissements peuvent être longs
        if 0.5 <= duration <= 60.0:
            curr['end'] = nxt['start']
            curr['duration'] = round(duration, 3)
            final_dataset.append(curr)

    # Sauvegarde
    with open(JSON_OUTPUT, 'w', encoding='utf-8') as f:
        json.dump(final_dataset, f, indent=4, ensure_ascii=False)

    print(f"Succès ! {len(final_dataset)} segments sauvegardés dans :")
    print(f"   -> {JSON_OUTPUT}")

    # Vérification visuelle
    if len(final_dataset) > 0:
        print("\nExemple corrigé (regardez le champ 'text') :")
        # On cherche un segment qui contient une parenthèse pour vérifier
        for seg in final_dataset:
            if '(' in seg['text']:
                print(json.dumps(seg, indent=4, ensure_ascii=False))
                break

# Lancer la correction
xml_to_json_segments(XML_FILE)

In [ ]:
import json
import librosa
import soundfile as sf
import pandas as pd
from tqdm import tqdm

# Config
# Le fichier JSON créé précédemment
JSON_FILE = '/content/drive/MyDrive/segments_clean.json'
# Fichier audio source
AUDIO_FILE = '/content/drive/MyDrive/4_decembre.wav'
# Le dossier pour petits fichiers wav (segments)
OUTPUT_DIR = '/content/drive/MyDrive/Dataset_Assemblee_CTC'

# Création du dossier si nécessaire
os.makedirs(OUTPUT_DIR, exist_ok=True)

def create_audio_dataset_from_json():
    """
    Lit le JSON, charge l'audio, découpe et sauvegarde les segments.
    Génère aussi le CSV final pour l'entraînement.
    """

    # 1. Vérification du fichier JSON
    if not os.path.exists(JSON_FILE):
        print(f"ERREUR : Le fichier JSON {JSON_FILE} n'existe pas !")
        print("   -> Avez-vous bien lancé la fonction xml_to_json_segments() avant ?")
        return

    # 2. Chargement des segments depuis le JSON
    print(f"Chargement du JSON : {JSON_FILE}")
    with open(JSON_FILE, 'r', encoding='utf-8') as f:
        segments = json.load(f)
    print(f"{len(segments)} segments chargés à traiter.")

    # 3. Chargement de l'audio complet
    print(f"Chargement du fichier audio complet en mémoire (16kHz)... Cela peut prendre 1 à 2 minutes.")
    # On force le chargement en 16000 Hz car c'est le standard pour CTC/ASR
    try:
        y, sr = librosa.load(AUDIO_FILE, sr=16000)
        print(f"Audio chargé ! Durée totale : {len(y)/sr:.2f} secondes.")
    except Exception as e:
        print(f"ERREUR lors du chargement de l'audio : {e}")
        return

    csv_data = []

    print("Découpage et sauvegarde des segments audio...")
    # tqdm pour voir la progression
    for i, seg in enumerate(tqdm(segments)):
        try:
            # Calcul des index de début et fin
            start_sample = int(seg['start'] * sr)
            end_sample = int(seg['end'] * sr)

            # Extraction du bout d'audio
            # Vérification pour ne pas dépasser la taille du fichier
            if end_sample > len(y):
                end_sample = len(y)

            clip = y[start_sample:end_sample]

            # Sauvegarde du petit fichier wav
            filename = f"segment_{i:05d}.wav"
            path = os.path.join(OUTPUT_DIR, filename)
            sf.write(path, clip, sr)

            # On ajoute les infos pour le CSV
            csv_data.append({
                "filename": filename,
                "text": seg['text'],
                "speaker": seg['speaker'],
                "duration": seg['duration']
            })

        except Exception as e:
            print(f"Erreur sur le segment {i} : {e}")
            continue

    # 4. Création du fichier Metadata (CSV)
    df = pd.DataFrame(csv_data)
    csv_path = os.path.join(OUTPUT_DIR, 'metadata.csv')
    df.to_csv(csv_path, index=False)

    print(f"\nTRAITEMENT TERMINÉ")
    print(f"   -> {len(df)} fichiers audio créés dans : {OUTPUT_DATASET_DIR}")
    print(f"   -> Fichier index (CSV) créé : {csv_path}")
    print("\n   Aperçu du CSV :")
    print(df.head())

# Lancement
create_audio_dataset_from_json()

Dans la cellule précédente on a eu un problème de décallage

Connexion au drive

In [ ]:
from google.colab import drive

# 1. Connexion au Google Drive
drive.mount('/content/drive')
print("Google Drive connecté.")

In [ ]:
Whisper

In [ ]:
import json
import os
import soundfile as sf
import librosa
import pandas as pd
import numpy as np
from tqdm import tqdm

# Config
# Fichiers
JSON_FILE = '/content/drive/MyDrive/segments_perfect.json'
AUDIO_FILE = '/content/drive/MyDrive/4_decembre.wav'

# Dossier de sortie
OUTPUT_DIR = '/content/drive/MyDrive/Dataset_Assemblee_Final'
WAVS_DIR = os.path.join(OUTPUT_DIR, 'wavs')

# Paramètres audio cibles pour l'entraînement
TARGET_SR = 16000 # 16kHz est le standard
Target_CHANNELS = 1 # Mono

# Création de l'arborescence
os.makedirs(WAVS_DIR, exist_ok=True)

def cut_and_save_dataset():
    # 1. Vérifications
    if not os.path.exists(JSON_FILE):
        print(f"ERREUR : Le JSON {JSON_FILE} n'existe pas.")
        return

    print(f"Lecture du JSON : {JSON_FILE}")
    with open(JSON_FILE, 'r', encoding='utf-8') as f:
        segments = json.load(f)
    print(f"{len(segments)} segments à traiter.")

    # 2. Analyse du fichier audio source
    print("Analyse du fichier audio source...")
    try:
        info = sf.info(AUDIO_FILE)
        native_sr = info.samplerate
        print(f"   -> Source : {info.duration/3600:.2f} heures @ {native_sr}Hz")
    except Exception as e:
        print(f"Erreur lecture audio : {e}")
        return

    csv_data = []

    print("Découpage, Rééchantillonnage (16k) et Sauvegarde...")

    # Barre de progression
    for i, seg in enumerate(tqdm(segments)):
        try:
            # Calcul des bornes
            start_sec = seg['start']
            end_sec = seg['end']
            duration = end_sec - start_sec

            # Sécurité : on ignore les segments vides ou aberrants
            if duration < 0.1: continue

            # Conversion en frames (échantillons) pour soundfile
            start_frame = int(start_sec * native_sr)
            frames_to_read = int(duration * native_sr)

            # Lecture stream
            # On lit seulement le petit morceau nécessaire
            audio_clip, _ = sf.read(AUDIO_FILE, start=start_frame, frames=frames_to_read, always_2d=True)

            # Traitement audio
            # 1. Conversion Stéréo -> Mono (Moyenne des canaux)
            if audio_clip.shape[1] > 1:
                audio_clip = np.mean(audio_clip, axis=1)
            else:
                audio_clip = audio_clip.flatten()

            # 2. Rééchantillonnage vers 16kHz (si nécessaire)
            if native_sr != TARGET_SR:
                # librosa.resample est efficace sur des petits clips
                audio_clip = librosa.resample(audio_clip, orig_sr=native_sr, target_sr=TARGET_SR)

            # Sauvegarde
            filename = f"segment_{i:05d}.wav"
            file_path = os.path.join(WAVS_DIR, filename)

            # On écrit le fichier propre (16k, mono)
            sf.write(file_path, audio_clip, TARGET_SR)

            # Metadata
            # On ajoute les infos pour le CSV final
            csv_data.append({
                "filename": filename,          # Nom du fichier (relatif)
                "text": seg['text'],           # Le texte de Whisper
                "speaker": seg['speaker'],     # L'orateur du XML
                "duration": round(len(audio_clip)/TARGET_SR, 2) # Durée réelle
            })

        except Exception as e:
            print(f"Erreur sur le segment {i} : {e}")
            continue

    # 3. Création du fichier Index (CSV)
    df = pd.DataFrame(csv_data)
    csv_path = os.path.join(OUTPUT_DIR, 'metadata.csv')
    df.to_csv(csv_path, index=False)

    print(f"\nDATASET FINAL TERMINÉ !")
    print(f"Dossier : {OUTPUT_DIR}")
    print(f"CSV Index : {csv_path}")
    print(f"Nombre d'exemples : {len(df)}")
    print("\nAperçu des données :")
    print(df.head())

# Lancement
cut_and_save_dataset()

Envoie sur le repo

In [3]:
from google.colab import drive
import shutil
import os

# accès au drive
drive.mount('/content/drive')

nom_du_notebook = "Notebook_Qui_Parle.ipynb"

chemin_source = f"/content/drive/My Drive/Colab Notebooks/{nom_du_notebook}"

# Nom du dossier Git (celui cloné précédemment)
nom_dossier_git = "esmeml/parliament_speaker_diarisation"
# -------------------------------

# 2. Vérification que le fichier source existe
if not os.path.exists(chemin_source):
    print(f"ERREUR : Le fichier n'a pas été trouvé à l'endroit : {chemin_source}")
    print("Vérifiez le nom du fichier ou s'il est dans un sous-dossier de votre Drive.")
else:
    # 3. Copie du notebook vers le dossier Git
    destination = f"/content/{nom_dossier_git}/{nom_du_notebook}"
    shutil.copy(chemin_source, destination)
    print(f"Succès : Le notebook a été copié dans le dossier Git !")

    # 4. Envoi vers GitHub
    # Changement du repertoire de travail
    os.chdir(f"/content/{nom_dossier_git}")

    print("Envoi vers GitHub en cours...")
    !git add "{nom_du_notebook}"  # On ajoute spécifiquement le notebook
    !git commit -m "Mise à jour du notebook de déploiement"
    !git push origin main # Ou 'master' selon votre branche

    print("Terminé ! Le notebook est maintenant sur le repo.")

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/esmeml/parliament_speaker_diarisation/Notebook_Qui_Parle.ipynb'